In [1]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import cv2 
import matplotlib
matplotlib.use('tkAgg')
import matplotlib.pyplot as plt
import math
import os,sys
from scipy.linalg import svd
import scipy.optimize
import glob

In [3]:
def get_H(save_directory, images_list, world_points):
    copy_images = images_list.copy()
    H_matrices = []
    image_points = []

    for i, image in enumerate(copy_images):
        grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        ret, corners_list = cv2.findChessboardCorners(grayscale, (9, 6), None)
        
        if ret:
            corners = corners_list.reshape(-1, 2)
            H_matrix, _ = cv2.findHomography(world_points, corners, cv2.RANSAC, 5.0)
            H_matrices.append(H_matrix)
            image_points.append(corners)

            cv2.drawChessboardCorners(image, (9, 6), corners, True)
            resized_image = cv2.resize(image, (image.shape[1] // 3, image.shape[0] // 3))
            output_path = os.path.join(save_directory, f'{i}_corners.png')
            cv2.imwrite(output_path, resized_image)
        else:
            print(f"Chessboard not found in image {i}")
    
    return H_matrices, image_points

In [4]:
import os
import cv2
import glob
import numpy as np

# Load images
image_files = glob.glob('calibration_images/*.jpg')
images = [cv2.imread(f) for f in image_files]

# Save path for output images
Save_path = 'output_images'
os.makedirs(Save_path, exist_ok=True)

# Create world coordinates for a 9x6 chessboard
objp = np.zeros((9*6, 2), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
world_coordinates = objp

# Call your function
H_total, corner_list = get_H(Save_path, images, world_coordinates)

In [5]:
import numpy as np

def get_K(H_total):
    # Construct V matrix as in Zhang's method for camera calibration
    def v_pq(H, p, q):
        return np.array([
            H[0, p]*H[0, q],
            H[0, p]*H[1, q] + H[1, p]*H[0, q],
            H[1, p]*H[1, q],
            H[2, p]*H[0, q] + H[0, p]*H[2, q],
            H[2, p]*H[1, q] + H[1, p]*H[2, q],
            H[2, p]*H[2, q]
        ])

    V = []
    for H in H_total:
        V.append(v_pq(H, 0, 1))
        V.append(v_pq(H, 0, 0) - v_pq(H, 1, 1))
    V = np.array(V)

    # Solve Vb=0 using SVD
    U, S, VT = np.linalg.svd(V)
    b = VT[-1, :]

    # Build B matrix
    B = np.array([
        [b[0], b[1], b[3]],
        [b[1], b[2], b[4]],
        [b[3], b[4], b[5]]
    ])

    # Compute intrinsic parameters using B
    try:
        v0 = (B[0,1]*B[0,2] - B[0,0]*B[1,2]) / (B[0,0]*B[1,1] - B[0,1]**2)
        lam = B[2,2] - (B[0,2]**2 + v0*(B[0,1]*B[0,2] - B[0,0]*B[1,2])) / B[0,0]

        print("B matrix:\n", B)
        print("v0:", v0)
        print("lambda:", lam)

        if lam <= 0:
            print("Warning: lambda is non-positive, calibration unstable.")
            return None  # or some fallback matrix

        alpha = np.sqrt(lam / B[0,0])
        beta = np.sqrt(lam * B[0,0] / (B[0,0]*B[1,1] - B[0,1]**2))
        gamma = -B[0,1]*alpha**2 * beta / lam
        u0 = gamma * v0 / beta - B[0,2]*alpha**2 / lam

        K = np.array([
            [alpha, gamma, u0],
            [0,     beta,  v0],
            [0,     0,     1]
        ])

        return K

    except Exception as e:
        print("Exception during intrinsic computation:", e)
        return None

In [6]:
import numpy as np

def get_R(K, H_total):
    R_matrices = []
    K_inv = np.linalg.inv(K)

    for H in H_total:
        # Compute lambda (scale factor)
        lam = 1 / np.linalg.norm(K_inv @ H[:, 0])

        # Compute r1 and r2 (rotation columns)
        r1 = lam * (K_inv @ H[:, 0])
        r2 = lam * (K_inv @ H[:, 1])

        # Compute r3 as cross product of r1 and r2
        r3 = np.cross(r1, r2)

        # Stack to form approximate rotation matrix
        R_approx = np.stack((r1, r2, r3), axis=1)

        # Check for NaNs or Infs in R_approx before SVD
        if np.any(np.isnan(R_approx)) or np.any(np.isinf(R_approx)):
            print("Warning: Invalid values in R_approx matrix, skipping this homography.")
            continue

        # Perform SVD to orthonormalize R_approx
        try:
            U, _, Vt = np.linalg.svd(R_approx)
        except np.linalg.LinAlgError:
            print("Warning: SVD did not converge for a rotation matrix, skipping.")
            continue

        R = U @ Vt

        # Ensure determinant is +1 for a proper rotation matrix
        if np.linalg.det(R) < 0:
            R *= -1

        R_matrices.append(R)

    return R_matrices

In [7]:
def get_loss_function(K_params, corner_list_all, world_coordinates, R_matrices):
    alpha, gamma, beta, uo, vo, k1, k2 = K_params
    f_errors = []

    for i, corners in enumerate(corner_list_all):
        Rt = R_matrices[i]

        # Check if Rt is 3x4, if not, raise a clear error
        if Rt.shape != (3, 4):
            raise ValueError(f"Expected Rt to be 3x4, got {Rt.shape}")

        total_error = 0.0  # ✅ Safe initialization

        for j, world_point in enumerate(world_coordinates):
            M = np.array([world_point[0], world_point[1], 0, 1])  # 3D point (Z=0)
            cam_coords = Rt @ M  # Apply extrinsics

            x, y, z = cam_coords.flatten()
            if z == 0:
                continue  # Skip if invalid projection

            x_norm = x / z
            y_norm = y / z

            r2 = x_norm**2 + y_norm**2
            x_dist = x_norm * (1 + k1 * r2 + k2 * r2**2)
            y_dist = y_norm * (1 + k1 * r2 + k2 * r2**2)

            u = alpha * x_dist + gamma * y_dist + uo
            v = beta * y_dist + vo

            predicted = np.array([u, v])
            actual = corners[j].reshape(2)
            error = np.linalg.norm(predicted - actual)

            total_error += error

        if len(world_coordinates) > 0:
            mean_error = total_error / len(world_coordinates)
        else:
            mean_error = 0  # Fallback default

        f_errors.append(mean_error)

    return np.array(f_errors)

In [8]:
import numpy as np
import scipy.optimize

def get_optimised(K, corner_list, world_coordinates, R):
    alpha, gamma, beta = K[0, 0], K[0, 1], K[1, 1]
    uo, vo = K[0, 2], K[1, 2]
    k1, k2 = 0.0, 0.0
    x0 = [alpha, gamma, beta, uo, vo, k1, k2]

    lower_bounds = [1e-3, -1e2, 1e-3, 0, 0, -1, -1]
    upper_bounds = [1e5,  1e2,  1e5,  1e4, 1e4, 1, 1]

    # Check and print any violations
    for i, (val, lb, ub) in enumerate(zip(x0, lower_bounds, upper_bounds)):
        if not (lb <= val <= ub):
            print(f"⚠️ x0[{i}] = {val} is out of bounds [{lb}, {ub}]")

    # Clamp values to be inside the bounds
    x0 = np.clip(x0, lower_bounds, upper_bounds)
    print("🔧 Clamped initial guess (x0):", x0)

    optimized = scipy.optimize.least_squares(
        fun=get_loss_function,
        x0=x0,
        method='trf',
        bounds=(lower_bounds, upper_bounds),
        args=(corner_list, world_coordinates, R)
    )

    alpha_u, gamma_u, beta_u, uo_u, vo_u, k1_u, k2_u = optimized.x
    K_updated = np.array([
        [alpha_u, gamma_u, uo_u],
        [0,       beta_u,  vo_u],
        [0,       0,       1]
    ])
    return K_updated, k1_u, k2_u
    
    fx, fy, cx, cy, k1, k2 = res.x
    K_opt = np.array([[fx, 0, cx],
                      [0, fy, cy],
                      [0,  0,  1]])
    return K_opt, (k1, k2), extrinsics

In [9]:
def get_error(updated_K, updated_K_distorted, corner_list, world_coordinates, R):
    uo = updated_K[0, 2]
    vo = updated_K[1, 2]
    k1, k2 = updated_K_distorted

    total_error = 0
    reprojection_errors_per_image = []
    proj_points = []

    for i, corners in enumerate(corner_list):
        K_ = updated_K @ R[i]
        error_sum = 0
        projected_corners = []

        for j in range(world_coordinates.shape[0]):
            world_point = world_coordinates[j]
            # Make homogeneous coordinate (assuming z=0)
            M = np.array([world_point[0], world_point[1], 1]).reshape(3, 1)

            image_coords = R[i] @ M
            x, y = image_coords[0] / image_coords[2], image_coords[1] / image_coords[2]

            projected_coords = K_ @ M
            u = projected_coords[0] / projected_coords[2]
            v = projected_coords[1] / projected_coords[2]

            corner_point = corners[j]
            corner_point_h = np.array([corner_point[0], corner_point[1], 1])

            # Apply radial distortion correction
            r2 = x**2 + y**2
            u_hat = u + (u - uo) * (k1 * r2 + k2 * r2**2)
            v_hat = v + (v - vo) * (k1 * r2 + k2 * r2**2)

            projected_corners.append([u_hat, v_hat])

            u_hat = float(u_hat)
            v_hat = float(v_hat)

            corrected_corner = np.array([u_hat, v_hat, 1])
            error = np.linalg.norm(corner_point_h - corrected_corner)
            error_sum += error

        reprojection_errors_per_image.append(error_sum / world_coordinates.shape[0])
        total_error += error_sum
        proj_points.append(projected_corners)

    avg_error = total_error / (len(corner_list) * world_coordinates.shape[0])
    return avg_error, proj_points

In [10]:
import cv2
import matplotlib.pyplot as plt

for img_path, corners in zip(image_files, corner_list):
    img = cv2.imread(img_path)
    img_corners = cv2.drawChessboardCorners(img.copy(), (pattern_width, pattern_height), corners, True)
    plt.imshow(cv2.cvtColor(img_corners, cv2.COLOR_BGR2RGB))
    plt.title(f'Corners detected in {img_path}')
    plt.show()

In [12]:
import cv2
import numpy as np
import glob
import os

def compute_homography(world_points, image_points):
    """
    Compute homography matrix from world points to image points.

    Parameters:
    - world_points: Nx2 numpy array of (x,y) points in world coordinates
    - image_points: Nx2 numpy array of corresponding (x,y) points in image coordinates

    Returns:
    - H: 3x3 homography matrix (numpy array)
    """
    H, status = cv2.findHomography(world_points, image_points, method=0)
    return H

def main():
    image_path = r'C:\Users\sripa\Downloads\Calibration Images'
    save_path = image_path  # Save results back in the same folder

    pattern_size = (9, 6)  # Chessboard inner corners (width x height)
    square_size = 21.5     # Size of a square in mm or your unit

    # Generate world coordinates for corners in checkerboard pattern
    world_x, world_y = np.meshgrid(range(pattern_size[0]), range(pattern_size[1]))
    world_coordinates = np.hstack((
        world_x.reshape(-1, 1),
        world_y.reshape(-1, 1)
    )).astype(np.float32) * square_size

    images = []
    corner_list = []
    H_total = []

    # Load images and detect corners
    image_files = glob.glob(os.path.join(image_path, '*.png'))
    if not image_files:
        raise ValueError(f"No PNG images found in {image_path}")

    for img_path in image_files:
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Warning: Unable to read image {img_path}, skipping.")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        found, corners = cv2.findChessboardCorners(gray, pattern_size)

        if found:
            # Refine corner locations for higher accuracy
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            corners_subpix = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

            print(f"✅ Detected corners in {os.path.basename(img_path)}: {len(corners_subpix)} (expected: {pattern_size[0]*pattern_size[1]})")

            # Compute homography from world coordinates to image corners
            H = compute_homography(world_coordinates, corners_subpix.reshape(-1, 2))
            print(f"Homography matrix:\n{H}\n")

            H_total.append(H)
            corner_list.append(corners_subpix.reshape(-1, 2))
            images.append(img)

            # Optional visualization
            debug_img = img.copy()
            cv2.drawChessboardCorners(debug_img, pattern_size, corners_subpix, found)
            cv2.imshow("Detected Checkerboard", debug_img)
            cv2.waitKey(300)
        else:
            print(f"⚠️ Checkerboard not detected in {os.path.basename(img_path)}")

    cv2.destroyAllWindows()

    if len(images) < 3:
        raise ValueError("Insufficient checkerboard images detected. Need at least 3 for calibration.")

    print(f"Total valid images for calibration: {len(images)}")
    print(f"Total homographies computed: {len(H_total)}")

    # Proceed with calibration using your existing functions
    K = get_K(H_total)
    if K is None:
        raise ValueError("Calibration failed: intrinsic matrix could not be computed. Check homographies and corner detection.")

    R = get_R(K, H_total)

    # Create 3x4 [R | t] extrinsic matrices
    R_matrices = []
    for R_single in R:
        t = np.zeros((3, 1))  # You can replace this with estimated translations if available
        Rt = np.hstack((R_single, t))
        R_matrices.append(Rt)

# Now pass the list of 3x4 Rt matrices to the optimizer
    K_updated, k1, k2 = get_optimised(K, corner_list, world_coordinates, R_matrices)
    distortion_updated = np.array([k1, k2]).reshape(2, 1)
    R_updated = get_R(K_updated, H_total)

    reprojected_error, reprojected_points = get_error(K_updated, distortion_updated, corner_list, world_coordinates, R_updated)

    K_final = np.array(K_updated, np.float32).reshape(3, 3)

    # Safely extract k1 and k2 as scalars
    k1 = float(distortion_updated[0]) if hasattr(distortion_updated[0], '__len__') else distortion_updated[0]
    k2 = float(distortion_updated[1]) if hasattr(distortion_updated[1], '__len__') else distortion_updated[1]

# Now create the distortion array with 4 coefficients: [k1, k2, 0, 0]
    distortion = np.array([k1, k2, 0.0, 0.0], dtype=np.float32)

    print("\n=== Calibration Results ===")
    print("Intrinsic Camera Matrix (K):\n", K_final)
    print("Distortion Coefficients:", distortion.ravel())
    print("Reprojection Error:", reprojected_error)

    os.makedirs(save_path, exist_ok=True)

    # Save undistorted and reprojected images
    for i, image_points in enumerate(reprojected_points):
        image = cv2.undistort(images[i], K_final, distortion)
        for point in image_points:
            image = cv2.circle(image, (int(point[0]), int(point[1])), 5, (128, 0, 128), 6)

        filename = os.path.join(save_path, f"{i}_reprojected_image.png")
        success = cv2.imwrite(filename, image)
        if success:
            print(f"✅ Saved undistorted image: {filename}")
        else:
            print(f"❌ Failed to save image: {filename}")

if __name__ == "__main__":
    main()

⚠️ Checkerboard not detected in 0_corners.png
⚠️ Checkerboard not detected in 1_corners.png
✅ Detected corners in 2_corners.png: 54 (expected: 54)
Homography matrix:
[[ 5.18886746e-01 -2.61061350e-02  1.25628871e+01]
 [ 2.80528603e-02  4.36334255e-01  1.25682932e+01]
 [ 2.11188181e-04 -2.11171590e-04  1.00000000e+00]]

⚠️ Checkerboard not detected in 3_corners.png
✅ Detected corners in 4_corners.png: 54 (expected: 54)
Homography matrix:
[[7.62276574e-01 1.33422751e-04 2.43555811e+01]
 [2.00556377e-03 7.57796585e-01 2.72270321e+01]
 [1.90504490e-05 1.43099907e-06 1.00000000e+00]]

✅ Detected corners in 5_corners.png: 54 (expected: 54)
Homography matrix:
[[7.62276574e-01 1.33422751e-04 2.43555811e+01]
 [2.00556377e-03 7.57796585e-01 2.72270321e+01]
 [1.90504490e-05 1.43099907e-06 1.00000000e+00]]

✅ Detected corners in 6_corners.png: 54 (expected: 54)
Homography matrix:
[[ 1.54712156e+00 -1.70075965e-04  3.30009819e+01]
 [ 1.17935012e-03  1.54636287e+00  3.28856387e+01]
 [-6.69319600e-06

C:\Users\sripa\AppData\Local\Temp\ipykernel_247208\1372192685.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u_hat = float(u_hat)
C:\Users\sripa\AppData\Local\Temp\ipykernel_247208\1372192685.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  v_hat = float(v_hat)
C:\Users\sripa\AppData\Local\Temp\ipykernel_247208\4127359177.py:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  k1 = float(distortion_updated[0]) if hasattr(distortion_updated[0], '__len__') el